# First Cell 

## makes sure the yfinance is the latest version and that yfinance, numpy and pandas are imported

In [ ]:
%pip install --upgrade yfinance

import yfinance as yf
import pandas as pd
import numpy as np

Note: you may need to restart the kernel to use updated packages.


### This cell defines the tickers and downloads them from Yfinance.

It also prints the first 5 lines of the data to prove that it was downloaded

In [ ]:


# Define the tickers, including the S&P 500
tickers = [ "AAPL","AMZN","GOOG","NKE","RY.TO","^GSPC"]  # RY.TO is for Royal Bank of Canada, ^GSPC is the S&P 500

# Download historical data (e.g., for the past year)
data = yf.download(tickers, period="1y")

print(data.head(5))



[*********************100%***********************]  6 of 6 completed

Price            Close                                                  \
Ticker            AAPL        AMZN        GOOG         NKE       RY.TO   
Date                                                                     
2024-02-20  180.706726  167.080002  141.690872  101.557899  127.835495   
2024-02-21  181.463181  168.589996  143.325012  103.494667  127.218254   
2024-02-22  183.503540  174.580002  144.799728  103.307877  127.594383   
2024-02-23  181.662216  174.990005  144.769821  103.848602  128.481659   
2024-02-26  180.308609  174.729996  138.253250  102.776985  127.613686   

Price                          High                                      ...  \
Ticker            ^GSPC        AAPL        AMZN        GOOG         NKE  ...   
Date                                                                     ...   
2024-02-20  4975.509766  181.572633  168.710007  142.815828  102.088786  ...   
2024-02-21  4981.799805  182.030494  170.229996  143.464510  103.740451  ...   
2024-02

## This cell calculates the daily returns based on the Closing price and also develops shows a correlation matrix

In [18]:
# --- Calculate Daily Returns ---
returns = data['Close'].pct_change()
print(returns.head(15))
# --- Calculate Correlations ---
correlation_matrix = returns.corr()
print(correlation_matrix)

Ticker          AAPL      AMZN      GOOG       NKE     RY.TO     ^GSPC
Date                                                                  
2024-02-20       NaN       NaN       NaN       NaN       NaN       NaN
2024-02-21  0.004186  0.009038  0.011533  0.019071 -0.004828  0.001264
2024-02-22  0.011244  0.035530  0.010289 -0.001805  0.002957  0.021123
2024-02-23 -0.010034  0.002349 -0.000207  0.005234  0.006954  0.000348
2024-02-26 -0.007451 -0.001486 -0.045013 -0.010319 -0.006756 -0.003787
2024-02-27  0.008114 -0.006811  0.009730  0.005835 -0.008313  0.001706
2024-02-28 -0.006625 -0.002190 -0.019058 -0.007608 -0.002210 -0.001658
2024-02-29 -0.003693  0.020790  0.017100 -0.004025  0.006645  0.005229
2024-03-01 -0.006030  0.008260 -0.012162 -0.016223  0.001062  0.008008
2024-03-04 -0.025381 -0.003591 -0.028100 -0.022968 -0.000227 -0.001193
2024-03-05 -0.028441 -0.019484 -0.003130 -0.012357  0.009173 -0.010193
2024-03-06 -0.005878 -0.003503 -0.009119 -0.006917 -0.002253  0.005141
2024-0

/var/folders/v7/k3z3dzlx5rv8j1g_txkst4vm0000gn/T/ipykernel_18171/3999015160.py:2: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns = data['Close'].pct_change()


## Calculate Volatility, Returns and Sharpe Ratio

This also is where the risk free rate is downloaded. It uses the ten year treasury bond rate


In [26]:


# --- Calculate Rolling Volatility (e.g., 30-day) ---
volatility = returns.rolling(window=30).std()

# --- Calculate Cumulative Returns ---
cumulative_returns = (1 + returns).cumprod() - 1

# --- Risk-Free Rate (Example: 10-year US Treasury yield - you'll want to update this) ---
risk_free_rate = yf.download("^TNX", period="1y")['Close'].iloc[-1] / 100 # Annualized risk-free rate.  Divide by 100 to convert percentage to decimal.
# Important: ^TNX is for the 10-year US Treasury.  You might want to use a more appropriate risk-free rate for your analysis (e.g., a Canadian government bond yield if analyzing Canadian stocks).

# --- Sharpe Ratio Calculation (Annualized) ---
def calculate_sharpe_ratio(returns, risk_free_rate, annualization_factor=252): # 252 trading days in a year
    excess_returns = returns - risk_free_rate/annualization_factor # Daily excess return
    annualized_excess_returns = excess_returns.mean() * annualization_factor # Annualize excess return
    annualized_volatility = volatility.mean() * np.sqrt(annualization_factor) # Annualize volatility
    sharpe_ratio = annualized_excess_returns / annualized_volatility
    return sharpe_ratio


sharpe_ratios = calculate_sharpe_ratio(returns, risk_free_rate)

#print("this is the volatiltiy")
#print(volatility.head(50-60))
#print( "this is the cum returns")
#print(cumulative_returns.head(50-60))
#print("this is the risk free rate")
#print( risk_free_rate.head(5))
print("this is the Sharpe ratio")
print(sharpe_ratios)


[*********************100%***********************]  1 of 1 completed

this is the Sharpe ratio
Ticker
AAPL    NaN
AMZN    NaN
GOOG    NaN
NKE     NaN
RY.TO   NaN
^GSPC   NaN
^TNX    NaN
dtype: float64


In [ ]:



# --- Export to Excel (separate sheets) ---
with pd.ExcelWriter("stock_analysis.xlsx") as writer:
    data['Adj Close'].to_excel(writer, sheet_name="Adj Close Prices")  # Adjusted Close Prices
    returns.to_excel(writer, sheet_name="Daily Returns")
    correlation_matrix.to_excel(writer, sheet_name="Correlation Matrix")
    volatility.to_excel(writer, sheet_name="Volatility")
    cumulative_returns.to_excel(writer, sheet_name="Cumulative Returns")
    pd.DataFrame(sharpe_ratios, columns=['Sharpe Ratio']).to_excel(writer, sheet_name="Sharpe Ratios") # Sharpe Ratios


print("Data and analysis saved to stock_analysis.xlsx")

# --- Example of Accessing Data (for us) ---
#print("\nCorrelation Matrix:")
#print(correlation_matrix)

#print("\nVolatility:")
#print(volatility)

#print("\nCumulative Returns:")
#print(cumulative_returns)

# --- Further Analysis (Examples for us) ---
# ... (as before)

# Example: Analyzing correlations with the S&P 500
sp500_corr = correlation_matrix["^GSPC"]
print("\nCorrelation with S&P 500:")
print(sp500_corr)

print("\nSharpe Ratios:")
print(sharpe_ratios)